In [24]:
import os
import pandas as pd

In [25]:
csv_files_path = "../../data/forms/csv"
output_csv_file = "../../generated/forms/data.csv"
invalid_data_csv_file = "../../generated/forms/invalid-data.csv"

csv_files = os.listdir(csv_files_path)
csv_files = [f for f in csv_files if f.endswith('.csv')]

outer_merged_df = None
inner_merged_df = None
id_key = 'Identifiant'


def group_set(row):
    identifier = row[id_key]

    if identifier.isnumeric():
        return "A" if int(identifier) % 2 != 0 else "B"

    return "Invalid"


for i, csv_file in enumerate(csv_files):
    file_path = os.path.join(csv_files_path, csv_file)
    df = pd.read_csv(file_path, header=0)

    duplicated = df[df.duplicated(subset=[id_key], keep=False)]
    duplicated_count = len(duplicated) / 2

    if duplicated_count > 0:
        print(f"⚠️ {csv_file}: {duplicated_count} duplicate(s), keep only firsts")
        print(duplicated[id_key])
        print("")

    df = df.drop_duplicates(subset=[id_key], keep="first")

    if id_key not in df.columns:
        print(f"⚠️ '{id_key}' not found in {csv_file}.")
        print(f"Available columns : {df.columns.tolist()}")
        continue

    if outer_merged_df is None:
        outer_merged_df = df
    else:
        outer_merged_df = pd.merge(outer_merged_df, df, on=id_key, how='outer')

    if inner_merged_df is None:
        inner_merged_df = df
    else:
        inner_merged_df = pd.merge(inner_merged_df, df, on=id_key, how='inner')

inner_merged_df.insert(1, "group", inner_merged_df.apply(group_set, axis=1))
outer_merged_df.insert(1, "group", outer_merged_df.apply(group_set, axis=1))


⚠️ achievements.csv: 1.0 duplicate(s), keep only firsts
20    86
41    86
Name: Identifiant, dtype: object

⚠️ demographic.csv: 2.0 duplicate(s), keep only firsts
33    86
39    56
42    86
43    56
Name: Identifiant, dtype: object



## Differences

In [33]:
inner_outer_merged_df = pd.merge(inner_merged_df, outer_merged_df, how='outer', indicator=True)

deleted_rows = inner_outer_merged_df[inner_outer_merged_df['_merge'] != 'both']

columns_to_keep = ["Identifiant", "group"] + list(deleted_rows.loc[:, deleted_rows.columns.str.contains("Horodateur")])

not_filled = deleted_rows[columns_to_keep].fillna("Not filled")
not_filled.to_csv(invalid_data_csv_file, index=False)
not_filled

,Identifiant,group,Achievements Horodateur,Demographic Horodateur,Leaderboard Horodateur,Satisfaction Horodateur
6,2,B,17/04/2025 10:07:05,17/04/2025 08:50:26,17/04/2025 09:31:29,Not filled
16,39,A,Not filled,17/04/2025 08:48:02,Not filled,Not filled
25,67,A,17/04/2025 09:33:46,17/04/2025 08:49:09,Not filled,Not filled
27,69,A,17/04/2025 09:33:44,Not filled,17/04/2025 10:09:46,Not filled
32,77,A,17/04/2025 09:34:04,17/04/2025 08:48:24,Not filled,Not filled
35,82,B,17/04/2025 10:09:09,Not filled,17/04/2025 09:32:49,Not filled
37,86,B,17/04/2025 09:50:29,17/04/2025 08:49:11,Not filled,Not filled
42,Selice,Invalid,Not filled,17/04/2025 08:48:26,Not filled,Not filled
43,hicarlie,Invalid,Not filled,17/04/2025 08:48:34,17/04/2025 09:30:56,Not filled


## Merged data

In [34]:
inner_merged_df

,Identifiant,group,Achievements Horodateur,Facilité d'utilisation du mode achievements,Avis sur l'utilisation des achievements,Ressenti lors de l'utilisation des achievements ?,Ressenti après avoir utilisé les achievements,J'ai été distrait·e à cause des achievements pendant que j'écrivais mes tests ?,Motivation à écrire des tests avec le système d'achievements,Fréquence de consultation des achievements durant l'évaluation,...,"Clarté des règles du jeu (achievements, leaderboard)",Design et intégration dans IntelliJ,"Selon vous, quel mode est le plus fun ?",Le système de points et de progression vous a-t-il semblé juste ?,"J'ai été tenté d'écrire plus de tests pour augmenter mes points, même si les tests ne sont pas spécialement pertinents","Seriez-vous intéressé·e par l'utilisation d'outils gamifiés durant certains cours ?\n(Vérification et Validation, Génie Logiciel, ..)","Quel mode j'ai envie d'utiliser dans un cours où je peux être confronté à du test logiciel ? \n(Vérification et Validation, Génie Logiciel, ..)",Donner 3 points positifs du plugin,Donner 3 points négatifs du plugin,Donner 3 améliorations à apporter au plugin
0,15,A,17/04/2025 09:32:19,4,5,2,4,2,4,J'ai consulté les achievements tout le long de...,...,3,5,1,Non,5,2,5,"Facilité d'utilisation, Design agréable, Nous ...","Le comptage des points peut être ""manipulé"", J...",Mettre plus d'explications pour les achieveme...
1,47,A,17/04/2025 09:32:40,2,4,2,4,3,4,1 à 5 fois,...,5,5,2,Non,2,5,5,"Pratique, pas compliqué a installé, motivant",Des gens trichants en réécrivant plusieurs foi...,"D’empêcher la triche, et rien de plus"
2,9,A,17/04/2025 09:32:41,4,4,2,5,3,5,6 à 10 fois,...,4,5,2,Oui,2,2,3,"Belle intégration, chouette concept, jolie UI",Le leaderboard ne s’est pas affiché pendant to...,"Bouton pour actualiser le leaderboard, amélior..."
3,23,A,17/04/2025 09:32:42,4,3,3,3,4,3,1 à 5 fois,...,3,3,5,Non,4,5,5,Donne un but (récompense) à la création de tes...,"Calcul des points à retravailler, j'ai reçu de...",Vérifier le calcul des points du leaderbord me...
4,81,A,17/04/2025 09:32:59,1,5,1,5,1,5,1 à 5 fois,...,3,5,5,Oui,5,5,5,Bien intégré dans Intelliji\nJ'aime vraiment b...,C'est frustrant d'être battu dans le leader bo...,Pouvoir voir les trophées des autres
5,3,A,17/04/2025 09:33:13,5,4,2,3,2,4,1 à 5 fois,...,4,5,5,Non,2,3,5,Intuitif\nSeamless\nUn peu addictif (dans le b...,Pas claire sur quoi compte\nFacile a exploiter...,Un always on display discret\nUn traqueur de p...
6,54,B,17/04/2025 09:33:22,1,2,3,2,1,3,6 à 10 fois,...,2,5,3,Oui,4,5,3,- Permet d’étudier en s’amusant \n- facile à u...,Je n’en vois aucun,Il est parfait pour moi
7,31,A,17/04/2025 09:33:24,2,4,2,3,4,4,J'ai consulté les achievements tout le long de...,...,5,5,5,Non,2,4,2,"Interactif, compétitif, divertissant","peut-être trop divertissant, notifications en ...",Peut-être évaluer la qualité des tests et avoi...
8,119,A,17/04/2025 09:33:37,2,2,2,3,1,1,Jamais,...,2,4,5,Oui,1,5,5,C’est encourageant,Il n’y a pas de consignes definies,Mettre plus de consignes
9,83,A,17/04/2025 09:33:39,4,4,2,4,3,3,J'ai consulté les achievements tout le long de...,...,4,4,4,Oui,3,2,5,"Belle interface, simple d'utilisation, les not...",Prend beaucoup de taille sur l'ide quand ouver...,"Enlever les possibilités de triche, ajouter un..."


## Save to csv

In [5]:
inner_merged_df.to_csv(output_csv_file, index=False)
print(f"Merged forms data save at {output_csv_file}")

Merged forms data save at ./generated/forms/data.csv
